In [1]:
#!pip install transformers
import os
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
#from google.colab import drive
import textwrap
import progressbar
import keras
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import json

from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
from transformers import XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig
from transformers import XLMForSequenceClassification, XLMTokenizer, XLMConfig
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig

In [2]:
model = RobertaForSequenceClassification.from_pretrained("./Finetuned/mini_RoBERTa/")
device = torch.device('cuda')
output_dir = "./Finetuned/mini_RoBERTa/"
tokenizer = RobertaTokenizer.from_pretrained(output_dir)
model.to(device)

Some weights of the model checkpoint at ./Finetuned/mini_RoBERTa/ were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [3]:
from nltk.tokenize import sent_tokenize

MAX_LEN = 512

def mask_sentence(sent):
    return [int(token_id > 0) for token_id in sent]

def generate_docs_for_sentence(dataf, tokenizer, model,all_docs):
  all_docs = []
  for i in progressbar.progressbar(range(len(dataf['text']))):
    text = dataf['text'].iloc[i]
    sents = sent_tokenize(text)
    
    tok_sents = [tokenizer.tokenize(text, add_prefix_space=True) for text in sents]
    
    if(len(tok_sents) > MAX_LEN):
      toks = toks[len(toks)-MAX_LEN:]
    
    for sentence in tok_sents:
        if(len(sentence) > 510):
            sentence = sentence[:510]
    CLS = tokenizer.cls_token
    SEP = tokenizer.sep_token
    encoded_sents = [tokenizer.convert_tokens_to_ids([CLS] + sen + [SEP]) for sen in tok_sents]

    #encoded_sents = [[CLS] + sent + [SEP] for sent in encoded_sents]
    encoded_sents = pad_sequences(encoded_sents, maxlen=512,value=0, dtype="long", padding="post")
    
    att_masks = [mask_sentence(sent) for sent in encoded_sents]
    
    embeddings = []
    
    inputs = torch.tensor(encoded_sents)
    masks = torch.tensor(att_masks)
    i = 0
    for sent,mask in zip(encoded_sents,att_masks):
        i+=1
        if(i % 20 == 0):
            print(f"Building... {i} sentence")
            torch.cuda.empty_cache()
        inputs = torch.tensor(sent).unsqueeze(0)
        masks = torch.tensor(mask).unsqueeze(0)
        inputs.to(device)
        masks.to(device)
        outputs = model(input_ids=inputs, token_type_ids=None, attention_mask=masks,output_hidden_states=True)
        #print(outputs[0].shape())
    
        embedding_sent = outputs[1][0][0]
        #print(embedding_sent)
        #print(len(embedding_sent))
        embeddings.append(embedding_sent)
    
    #vecs = []
    #for sent in sentences
    #for index,ii in enumerate(splitted_input_ids):
    #  vecs.append(get_output_for_one_vec(ii, splitted_att_masks[index]))
 
    one_doc = ((embeddings,dataf['label'].iloc[i]))
    all_docs.append(one_doc)

  return all_docs

In [5]:
mini_dataset_path = "./Data/ILDC_single.csv" 
df = pd.read_csv(mini_dataset_path)
train_set = df.query(" split=='train' ")
test_set = df.query(" split=='test' ")
validation_set = df.query(" split=='dev' ")

In [6]:
all_docs = []
docs = generate_docs_for_sentence(df,tokenizer,model,all_docs)


  0% (0 of 7593) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)